In [19]:
import torch
import numpy as np
from scipy import sparse
from Utils.layers import CITRUS
import networkx as nx

In [20]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


The next function gives the eigenvector of a Cartesian product graph $\mathcal{G}_\diamond=\mathcal{G}_1\oplus...\oplus\mathcal{G}_P$ and also the list of egienvalues of the factor graphs as $\mathbf{V}_\diamond=\mathbf{V}_1\otimes...\otimes \mathbf{V}_P$ and $\lambda_{list}=[\boldsymbol{\lambda}_1,...,\boldsymbol{\lambda}_P]$, where the EVD form of the $p$-th factor Laplacian:
$\mathbf{L}_p=\mathbf{V}_p diag(\boldsymbol{\lambda}_p)\mathbf{V}^\top_p$ for $p=1,...,P$.

In [21]:
def get_selected_evec_evals(L_normalized_sparse_list, k_list):
    evals, evecs = sparse.linalg.eigs(L_normalized_sparse_list[0], k=k_list[0], return_eigenvectors=True)
    evals = torch.tensor(evals.real)
    evals = evals.to(torch.float32)
    evals_list = [evals]
    evecs=torch.tensor(evecs.real).to(torch.float32)        
    evecs_kron = evecs
    
    for p in range(1, len(L_normalized_sparse_list)):

        evals, evecs = sparse.linalg.eigs(L_normalized_sparse_list[p], k=k_list[p], return_eigenvectors=True)
        evals = torch.tensor(evals.real)
        evals = evals.to(torch.float32)
        evals_list.append(evals)
        evecs = torch.tensor(evecs.real)        
        evecs_kron = torch.kron(evecs_kron, evecs).to(torch.float32)
    
    return evals_list, evecs_kron

The next function gives a connected random Erdős-Rényi (ER) graph with the number of nodes $n$ and edge-probability of $p$.

In [22]:
def gen_connected_ER(n, p):
    connected = False
    while not connected:
        G = nx.erdos_renyi_graph(n, p)
        connected = nx.is_connected(G)
    return G

The nest function generates the factor ER graphs with the number of nodes $N_{list}$ and edge prpbblities of $p_{list}$.

In [23]:
def gen_factor_graphs(N_list, p_list):
    
    P = len(N_list)
    
    Adj_list = []
    
    G = gen_connected_ER(N_list[0], p_list[0])
    L = nx.laplacian_matrix(G).toarray()
    
    # Compute degree matrix
    A = nx.to_numpy_array(G)  
    Adj_list.append(A)
    degrees = np.sum(A, axis=1)
    # D = np.diag(degrees)
    
    # Compute normalized Laplacian
    D_sqrt_inv = np.diag(1.0 / np.sqrt(degrees))
    L_normalized = np.dot(np.dot(D_sqrt_inv, L), D_sqrt_inv)
    L_normalized = L_normalized/P
    L_sparse = sparse.coo_matrix(L_normalized)
    L_normalized_sparse_list = [L_sparse]
    
    
    for p in range(1, P):
        G = gen_connected_ER(N_list[p], p_list[p])
        L = nx.laplacian_matrix(G).toarray()

        # Compute degree matrix
        A = nx.to_numpy_array(G)
        Adj_list.append(A)
        degrees = np.sum(A, axis=1)
        # D = np.diag(degrees)

        # Compute normalized Laplacian
        D_sqrt_inv = np.diag(1.0 / np.sqrt(degrees))
        L_normalized = np.dot(np.dot(D_sqrt_inv, L), D_sqrt_inv)
        L_normalized = L_normalized/P
        L_sparse = sparse.coo_matrix(L_normalized)
        L_normalized_sparse_list.append(L_sparse)

        
    return L_normalized_sparse_list
#%%

The next function implements a way for performing $i$-th mode matricization operation on a given tensor $\underline{\mathbf{X}}\in\mathbb{R}^{N_1\times...\times N_i\times...\times N_P}$ as $\underline{\mathbf{X}}_{(i)}$, where $\underline{\mathbf{X}}_{(i)}\in\mathbb{R}^{N_i\times\tilde{N}_i}$ with $\tilde{N}_i=\prod_{p=1,p\ne i}^{P}{N_p}$. For more details, please refer to the tensor decompostion nad rocessing literature, like [1] and [2]. 

[1]  T. G. Kolda and B. W. Bader, “Tensor decompositions and applications,” SIAM Review, vol. 51,
no. 3, pp. 455–500, 2009.

[2] N. D. Sidiropoulos, L. De Lathauwer, X. Fu, K. Huang, E. E. Papalexakis, and C. Faloutsos,
“Tensor decomposition for signal processing and machine learning,” IEEE Transactions on
Signal Processing, vol. 65, no. 13, pp. 3551–3582, 2017.


In [24]:
def mode_n_matricization(tensor, mode):
    """
    Perform mode-n matricization of a tensor.

    Parameters:
    tensor (ndarray): The input tensor of shape (I1, I2, ..., IN).
    mode (int): The mode along which to matricize (0-based index).

    Returns:
    ndarray: The mode-n matricized tensor.
    """
    # Move the mode-th axis to the first dimension
    tensor = np.moveaxis(tensor, mode, 0)
    # Get new shape: (size of mode-n, product of remaining dimensions)
    new_shape = (tensor.shape[0], -1)
    # Reshape tensor to new shape
    matricized_tensor = tensor.reshape(new_shape)
    return matricized_tensor

The next function rearranges a given mode-$i$ matricized form of a tensor ($\underline{\mathbf{X}}_{(i)}$) to produce the initil tensor $\underline{\mathbf{X}}$.

In [25]:
def reverse_mode_n_matricization(matricized_tensor, original_shape, mode):
    """
    Reverse mode-n matricization to reconstruct the original tensor.

    Parameters:
    matricized_tensor (ndarray): The matricized tensor of shape (I_n, prod(I_1, ..., I_{n-1}, I_{n+1}, ..., I_N)).
    original_shape (tuple): The original shape of the tensor (I1, I2, ..., IN).
    mode (int): The mode along which the matricization was performed (0-based index).

    Returns:
    ndarray: The reconstructed tensor with the original shape.
    """
    # Determine the shape after expanding back to the original tensor's dimensions
    new_shape = (original_shape[mode],) + tuple(dim for i, dim in enumerate(original_shape) if i != mode)
    
    # Reshape the matricized tensor back to this expanded shape
    reshaped_tensor = matricized_tensor.reshape(new_shape)
    
    # Reverse the axis reordering to get back to the original shape
    reconstructed_tensor = np.moveaxis(reshaped_tensor, 0, mode)
    return reconstructed_tensor

Next, using some parameters, the the facor graphs, and their eigenvalue-eigenvector forms are generated.  

In [26]:
#%% Generate the graphs:
p_list = [0.3, 0.3, 0.05]
N_list = [10, 15, 20]
L_list = gen_factor_graphs(N_list, p_list)
k = np.array(N_list)-2
k_list = [N_list[0] - 2, N_list[1] - 2, N_list[2] - 2]
evals, evecs = get_selected_evec_evals(L_list, k_list)

Then, the parameters for generating the data tensor $\underline{\mathbf{X}}$ and building the CITRUS blocks are specified.

In [ ]:
Fea = [4, 2]
N_block = 32
C_width = 4

In the following, a sample data is generated first and then a CTRUS model with $N_{blocks}=32$ blocks is built to transform the (matricized) given data $\underline{\mathbf{X}}^\top_{(4)}\in\mathbb{R}^{[\overbrace{10\times15\times20}^{3000}]\times4}$ to the (matricized) output data $\underline{\mathbf{Y}}^\top_{(4)}\in\mathbb{R}^{[\overbrace{10\times15\times20}^{3000}]\times2}$, where $\underline{\mathbf{X}}\in\mathbb{R}^{10\times15\times20\times4}$ and $\underline{\mathbf{Y}}\in\mathbb{R}^{10\times15\times20\times2}$. For more details, please refer to our paper [3].

[3] Einizade, Aref, Fragkiskos D. Malliaros, and Jhony H. Giraldo. "Continuous Product Graph Neural Networks." arXiv preprint arXiv:2405.18877 (2024).

In [27]:
X_tensor = torch.randn(N_list[0], N_list[1], N_list[2], Fea[0])

print('X_tensor.shape: ', X_tensor.shape)

X_unfolded = torch.tensor(mode_n_matricization(X_tensor.numpy(), 3).T).to(device)

print('X_unfolded.shape: ', X_unfolded.shape)

## Run mode
            
model = CITRUS(k = np.prod(k), C_in = Fea[0], C_out = Fea[-1], C_width = C_width, num_nodes = N_list,
          N_block = N_block, single_t = False, use_gdc = [],
            last_activation = lambda x : x,
            diffusion_method = 'spectral',
            with_MLP = True,
            dropout = True,
            device = device)
            
model = model.to(device)


model_CPGNN = model
    
parameters = [p for p in model.parameters() if p.requires_grad]

parameters_name= [ name for (name, param) in model.named_parameters() if param.requires_grad]

# Move to device
mass=torch.ones(np.prod(N_list)).to(device)

for ii in range(len(evals)):
    evals[ii] = evals[ii].to(device)
evecs=evecs.to(device)

out_unfolded = model(0, X_unfolded, [], mass=mass, L=L_list, evals=evals, evecs=evecs) #GITHUUUUUUUUUB

print('out_unfolded.shape: ', out_unfolded.shape)

out_tensor = reverse_mode_n_matricization(out_unfolded.cpu().detach().numpy(), (N_list[0], N_list[1], N_list[2], Fea[-1]), 3)

print('out_tensor.shape: ', out_tensor.shape)


X_tensor.shape:  torch.Size([10, 15, 20, 4])
X_unfolded.shape:  torch.Size([3000, 4])
out_unfolded.shape:  torch.Size([3000, 2])
out_tensor.shape:  (10, 15, 20, 2)
